In [1]:
# !pip install optuna
# !pip install torchsummary

In [1]:
# Importing relevant packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import optuna

from project1_model_optuna import ResNet, BasicBlock


In [2]:
# fetching device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# worker and batch size init
num_workers = 10
batch_size = 128
pin_memory = True

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        # transforms.RandomVerticalFlip(),
        # transforms.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation='bilinear', fill=0),
        transforms.RandomRotation(degrees=(90)),
        transforms.RandomCrop(32, padding=4),
        #transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2471, 0.2435, 0.2616]),
        #transforms.RandomErasing(),

    ]),
    'val': transforms.Compose([
        # transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2471, 0.2435, 0.2616])
    ]),
}


dataloaders = {}
for phase in ['train','val']:
    dataset = datasets.CIFAR10(root='./data', train=(phase=='train'),download=True, transform=data_transforms[phase])
    dataloaders[phase] = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle=(phase=='train'), num_workers=num_workers,pin_memory=pin_memory)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
save_loss = {'train':[], 'val':[]}
save_acc = {'train':[], 'val':[]}

In [5]:
def train_and_evaluate(trial,model,criterion,optimizer,num_epochs=20):
    for epoch in range(num_epochs):
    # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            for batch_idx, (inputs, labels) in enumerate(dataloaders[phase], 1):
                inputs,labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                # Time to carry out the forward training poss
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # We want variables to hold the loss/acc statistics
                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)
            # saving variable for plottin
            save_loss[phase] += [current_loss / len(dataloaders[phase].dataset)]
            save_acc[phase] += [current_corrects.float() / len(dataloaders[phase].dataset)]
            # train_loss = train_loss/len(trainloader)
            # test_loss = test_loss/len(testloader)
            # train_loss_history.append(train_loss)
            # test_loss_history.append(test_loss)
            # print('Train loss %s, Test loss %s'%(train_loss, test_loss))
            # pretty print
#             print(f"Epoch:{epoch} -- Phase:{phase} -- Loss:{save_loss[phase][-1]:.2f} -- Acc:{save_acc[phase][-1]*100:.2f}")
    return max(save_acc['val'])*100


In [6]:
# optuna hyperparameter tuning tool
def objective(trial):

    params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2, 1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam","Adadelta"]),
              'weight_decay': trial.suggest_loguniform('weight_decay', 1e-4, 1e-3),
              'kernel_size': trial.suggest_categorical("kernel_size", [3,5]),
              'skip_kernel_size': trial.suggest_categorical("skip_kernel_size", [1,3,5])
              }
    # get model
    model = ResNet(BasicBlock, [4,2], 128, params['kernel_size'],params['skip_kernel_size'])
    model = model.to(device)
    
    # Define criterion
    criterion = nn.CrossEntropyLoss()
    
    # Configure optimizer
    optimizer = getattr(
        torch.optim, params["optimizer"]
    )(model.parameters(), lr=params["learning_rate"],weight_decay=params["weight_decay"])

    accuracy = train_and_evaluate(trial,model,criterion,optimizer,100)

    return accuracy

In [ ]:

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=300)

[I 2022-03-24 02:18:43,771] A new study created in memory with name: no-name-7ab41e59-f96b-4018-b861-ceace9f1c5aa
[I 2022-03-24 05:36:59,897] Trial 0 finished with value: 85.74999237060547 and parameters: {'learning_rate': 0.4204055980989263, 'optimizer': 'Adadelta', 'weight_decay': 0.0002027218637632411, 'kernel_size': 5, 'skip_kernel_size': 5}. Best is trial 0 with value: 85.74999237060547.


In [ ]:
# f, (plt1, plt2) = plt.subplots(1, 2, figsize=(20, 10))
# plt1.plot(save_acc['train'])
# plt1.plot(save_acc['val'])
# plt1.legend(["train", "val"])
# plt1.title("Accuracy")

# plt2.plot(save_loss['train'])
# plt2.plot(save_loss['val'])
# plt2.legend(["train", "val"])
# plt2.title("Loss")

In [1]:
import torch

torch.cuda.device_count()

1